In [166]:
import datetime as dt
import requests
import requests_cache
requests_cache.install_cache("bigdave", expire_after=dt.timedelta(days=1)) 
from lxml import html
import re
from time import sleep

In [167]:
s = requests.Session()

Want to start at the home page (maybe don't cache this).  
Find the first page named 'DT \d{5}'  

Go to that page.  
Crawl the clues.  
Find the nav links, follow them until the next DT page is found.  
Repeat until no more nav links?  
Be polite.  


In [168]:
base_url = "http://bigdave44.com"

In [169]:
r = s.get(base_url)
r.raise_for_status()

In [170]:
tree = html.fromstring(r.content)
links = tree.xpath('//h2[@class="entry-title"]/a[@rel="bookmark"]/@href')
print(links)

['http://bigdave44.com/2009/01/28/welcome/', 'http://bigdave44.com/2023/03/22/toughie-3030/', 'http://bigdave44.com/2023/03/22/dt-30254/', 'http://bigdave44.com/2023/03/22/sunday-toughie-59-review/', 'http://bigdave44.com/2023/03/22/st-3203-full-review/', 'http://bigdave44.com/2023/03/21/rip-pam-morton-mrs-bd/', 'http://bigdave44.com/2023/03/21/167421/', 'http://bigdave44.com/2023/03/21/dt-30253/', 'http://bigdave44.com/2023/03/20/dt-30252/', 'http://bigdave44.com/2023/03/20/rookie-corner-467/', 'http://bigdave44.com/2023/03/19/sunday-toughie-60-hints/']


In [171]:
for l in links:
    if not 'dt-' in l:
        continue
    print(f"Getting {l}")
    break
r = s.get(l)
r.raise_for_status()

Getting http://bigdave44.com/2023/03/22/dt-30254/


In [172]:
def get_clues(l, content):
    clue_pairs = []
    tree = html.fromstring(content)
    if re.search('/dt-\d+/', l):
        clues = tree.xpath('//p')
        for _c in clues:
            c = re.sub(r'\s+', ' ', " ".join(_c.itertext()))
            c = re.sub(r'[—-]', '-', c)
            if not re.match(r'\d+[ad]', c):
                continue
            # print(c)
            clue, answer = None, None
            # Y’S 
            if (re.search(r'\d+[ad]\s+(.+\s\([\d,—-]+\))', c)):
                if (re.search(r'\([\d,—-]+\)\s+([A-Z\s—-]+)\s?[:—-]', c, re.I)):
                    clue = re.search(r'\d+[ad]\s+(.+\s\([\d,—-]+\))', c).groups()
                    answer = re.search(r'\([\d,—-]+\)\s+([A-Z’\'\s—-]+)\s?[:—-]', c, re.I).groups()
                    clue_pairs.append((clue[0].strip(), answer[0].strip())) 
                else:
                    print("Didn't find answer:", c)
                    
            else:
                # print("Didn't find clue:", c)
                ...
            
            # print (clue, answer)
    links = tree.xpath('//div [@class="nav-previous"]/a')
    
    # print(links[0].get('href'))
    return(clue_pairs, links[0].get('href'))
        

In [173]:
pairs = []


Didn't find answer: 20a Have expertise - able to tell a scallion from a shallot? (4,4,6) KNOW ONE’S ONIONS - literally interpreted, this idiomatic expression could denote the ability to distinguish one variety of allium from another
Didn't find answer: 1a This could be blooming thing Cinderella lost? (5-7) LADY’S SLIPPER : This might be what Prince Charming found on the stairway. 
Didn't find answer: 29a Article revealing scandal with former model (6) EXPOSÉ : The two-letter term for former, and a synonym for model as a verb
Didn't find answer: 13a, 14a, and 15a give a (6,6) pun.
Didn't find answer: 19a Feeling over in market, disordered place (5-4) MARE’S-NEST - reverse a word meaning a general feeling or emotion and insert it into another term for market; although the BRB does not do so, the online Chambers 21st Century Dictionary indicates this to be a chiefly US usage
Didn't find answer: 12a Cured meat and tripe (6): DT 30156
Didn't find answer: 19a Frame piece of legislation intro

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [174]:
# Finished at http://bigdave44.com/2016/04/14/dt-28088/ 

crawl=True
crawled_links = []
max_pages = 1800 # approx five years' worth
while crawl:
    if re.search('/dt-\d+/', l):
        crawled_links.append(l)
        print(l, f"({len(crawled_links)})    ", end='\r')
    r = s.get(l)
    if not r.from_cache:
        # print("Sleeping")
        sleep(5)
    r.raise_for_status()
    _pairs, l = get_clues(l, r.content)
    
    pairs.extend(_pairs)
    if not l:
        crawl = False
    if len(crawled_links) > max_pages:
        crawl=False

Didn't find answer: 1a Jumping straight on funfair ride (5,5) GHOST TRAIN – Anagram (jumping) of STRAIGHT ON.
Didn't find answer: 6a Intimate function, close to capacity (4) COSY – The short form of a trigonometric function, followed by the last letter (close) of capacit Y .
Didn't find answer: 9a Ne’er-do-well wanting second something to drink (5) WATER – Remove the abbreviation for S econd from a word for a ne’er-do-well, to get a universal drink.
Didn't find answer: 10a Vivid report of selected Oxford college (9) PICTORIAL – There are two homophones (report of) here: a word for ‘selected’ followed by the name of an Oxford college.
Didn't find answer: 12a Passionate old travellers in retirement going to America (7) AMOROUS – Put together an abbreviation for O ld and the name of the traveller or gypsy people, reverse the result (in retirement), then add an abbreviation for America.
Didn't find answer: 13a Dance from Strictly in second place? (5) TANGO – The second letter of s T rictly

In [175]:
_str = " 17d Wisecrack from individual on ship (3-5) ONE-LINER — another word for individual and a large passenger ship"
re.search(r'\([\d,—-]+\)\s+([A-Z\s—-]+)\s?[:—-]', _str, re.I)


<re.Match object; span=(39, 56), match='(3-5) ONE-LINER —'>

In [176]:
len(pairs), pairs[0]

(44313,
 ('Giving affection, having a close relationship (4,2,5)', 'HAND IN GLOVE'))

In [177]:
with open('pairs.txt', 'w', encoding='utf-8') as outfile:
    for p in pairs:
        print("\t".join(p), file=outfile)

In [ ]:
365*5

1825